In [1]:
import pandas as pd
import os
from functools import reduce

In [2]:
dfNitrate = pd.read_parquet("../data/nitrate.parquet")
dfAmmonium = pd.read_parquet("../data/ammonium.parquet")
dfOxygenA = pd.read_parquet("../data/oxygen_a.parquet")
dfOxygenB = pd.read_parquet("../data/oxygen_b.parquet")
dfPhosphate = pd.read_parquet("../data/phosphate.parquet")
dfWater = pd.read_csv("../data/water.csv", delimiter=';')

In [3]:
dfNitrate.rename(columns={'hstWaarde': 'nitrate'}, inplace=True)
dfAmmonium.rename(columns={'hstWaarde': 'ammonium'}, inplace=True)
dfOxygenA.rename(columns={'hstWaarde': 'oxygena'}, inplace=True)
dfOxygenB.rename(columns={'hstWaarde': 'oxygenb'}, inplace=True)
dfPhosphate.rename(columns={'hstWaarde': 'phosphate'}, inplace=True)
dfWater.rename(columns={'EDE_09902MTW_K100.MTW': 'throughput', 'DateTime': 'datumEindeMeting'}, inplace=True)

In [4]:
dfWater

,datumEindeMeting,throughput,wwResolution
0,1-1-2021 00:01,"2754,761089",60000
1,1-1-2021 00:02,"2609,343947",60000
2,1-1-2021 00:03,"2535,541065",60000
3,1-1-2021 00:04,"2780,798536",60000
4,1-1-2021 00:05,"2192,832206",60000
...,...,...,...
525594,31-12-2021 23:55,"907,6167848",60000
525595,31-12-2021 23:56,"176,3518071",60000
525596,31-12-2021 23:57,"113,5575971",60000
525597,31-12-2021 23:58,"187,9508758",60000


In [5]:
dfWater['datumEindeMeting'] =  pd.to_datetime(dfWater['datumEindeMeting'], dayfirst=True)

In [6]:
dfWater

,datumEindeMeting,throughput,wwResolution
0,2021-01-01 00:01:00,"2754,761089",60000
1,2021-01-01 00:02:00,"2609,343947",60000
2,2021-01-01 00:03:00,"2535,541065",60000
3,2021-01-01 00:04:00,"2780,798536",60000
4,2021-01-01 00:05:00,"2192,832206",60000
...,...,...,...
525594,2021-12-31 23:55:00,"907,6167848",60000
525595,2021-12-31 23:56:00,"176,3518071",60000
525596,2021-12-31 23:57:00,"113,5575971",60000
525597,2021-12-31 23:58:00,"187,9508758",60000


In [7]:
dfAmmonium

,historianTagnummer,ammonium,datumBeginMeting,datumEindeMeting
0,EDE_B121069913_K600.MTW,4.487,2021-01-01 00:00:00,2021-01-01 00:01:00
1,EDE_B121069913_K600.MTW,4.263,2021-01-01 00:01:00,2021-01-01 00:02:00
2,EDE_B121069913_K600.MTW,4.173,2021-01-01 00:02:00,2021-01-01 00:03:00
3,EDE_B121069913_K600.MTW,4.194,2021-01-01 00:03:00,2021-01-01 00:04:00
4,EDE_B121069913_K600.MTW,4.214,2021-01-01 00:04:00,2021-01-01 00:05:00
...,...,...,...,...
44635,EDE_B121069913_K600.MTW,2.256,2021-12-31 23:55:00,2021-12-31 23:56:00
44636,EDE_B121069913_K600.MTW,2.264,2021-12-31 23:56:00,2021-12-31 23:57:00
44637,EDE_B121069913_K600.MTW,2.273,2021-12-31 23:57:00,2021-12-31 23:58:00
44638,EDE_B121069913_K600.MTW,2.281,2021-12-31 23:58:00,2021-12-31 23:59:00


In [8]:
dfs = [dfNitrate, dfAmmonium, dfOxygenB, dfPhosphate, dfWater]

In [9]:
dfs = [dfNitrate, dfAmmonium, dfOxygenB, dfPhosphate]
column_to_drop = 'historianTagnummer'

for df in dfs:
    if column_to_drop in df.columns:
        df.drop(column_to_drop, axis=1, inplace=True)

In [10]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['datumBeginMeting', 'datumEindeMeting'],
                                            how='inner'), dfs)

In [11]:
df_merged['datumBeginMeting'] = pd.to_datetime(df_merged['datumBeginMeting'])
df_merged['datumEindeMeting'] = pd.to_datetime(df_merged['datumEindeMeting'])

In [12]:
dfWater.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525599 entries, 0 to 525598
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   datumEindeMeting  525599 non-null  datetime64[ns]
 1   throughput        525599 non-null  object        
 2   wwResolution      525599 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 12.0+ MB


In [13]:
df_merged = df_merged.merge(dfWater, how='inner', on='datumEindeMeting')    

In [14]:
df_merged

,nitrate,datumBeginMeting,datumEindeMeting,ammonium,oxygenb,phosphate,throughput,wwResolution
0,6.14,2021-01-01 00:00:00,2021-01-01 00:01:00,4.487,0.029,0.185,"2754,761089",60000
1,6.118,2021-01-01 00:01:00,2021-01-01 00:02:00,4.263,0.028,0.126,"2609,343947",60000
2,5.963,2021-01-01 00:02:00,2021-01-01 00:03:00,4.173,0.028,0.108,"2535,541065",60000
3,5.884,2021-01-01 00:03:00,2021-01-01 00:04:00,4.194,0.028,0.122,"2780,798536",60000
4,5.876,2021-01-01 00:04:00,2021-01-01 00:05:00,4.214,0.028,0.136,"2192,832206",60000
...,...,...,...,...,...,...,...,...
218853,6.882,2021-12-31 23:54:00,2021-12-31 23:55:00,2.247,0.552,1.925,"907,6167848",60000
218854,6.89,2021-12-31 23:55:00,2021-12-31 23:56:00,2.256,0.549,1.935,"176,3518071",60000
218855,6.904,2021-12-31 23:56:00,2021-12-31 23:57:00,2.264,0.559,1.944,"113,5575971",60000
218856,6.985,2021-12-31 23:57:00,2021-12-31 23:58:00,2.273,0.554,2.092,"187,9508758",60000


In [15]:
dfPhosphate

,phosphate,datumBeginMeting,datumEindeMeting
0,0.185,2021-01-01 00:00:00,2021-01-01 00:01:00
1,0.126,2021-01-01 00:01:00,2021-01-01 00:02:00
2,0.108,2021-01-01 00:02:00,2021-01-01 00:03:00
3,0.122,2021-01-01 00:03:00,2021-01-01 00:04:00
4,0.136,2021-01-01 00:04:00,2021-01-01 00:05:00
...,...,...,...
44635,1.935,2021-12-31 23:55:00,2021-12-31 23:56:00
44636,1.944,2021-12-31 23:56:00,2021-12-31 23:57:00
44637,2.092,2021-12-31 23:57:00,2021-12-31 23:58:00
44638,2.133,2021-12-31 23:58:00,2021-12-31 23:59:00


In [18]:
dfPhosphate

,phosphate,datumBeginMeting,datumEindeMeting
0,0.185,2021-01-01 00:00:00,2021-01-01 00:01:00
1,0.126,2021-01-01 00:01:00,2021-01-01 00:02:00
2,0.108,2021-01-01 00:02:00,2021-01-01 00:03:00
3,0.122,2021-01-01 00:03:00,2021-01-01 00:04:00
4,0.136,2021-01-01 00:04:00,2021-01-01 00:05:00
...,...,...,...
44635,1.935,2021-12-31 23:55:00,2021-12-31 23:56:00
44636,1.944,2021-12-31 23:56:00,2021-12-31 23:57:00
44637,2.092,2021-12-31 23:57:00,2021-12-31 23:58:00
44638,2.133,2021-12-31 23:58:00,2021-12-31 23:59:00
